# Task #1

- Get all the tweets and dump into json files
- The text of the tweet.
- Date and time of the tweet. <br>
- The number of favorites/likes.<br>
- The number of retweets.<br>
- Number of Images present in Tweet. If no image returns None.

In [1]:
import tweepy
import pandas as pd
pd.options.display.max_columns = None
import warnings
warnings.filterwarnings('ignore')

import time

In [2]:
# CONSUMER KEYS
consumer_key = #---pase your consumer key here--#
consumer_secret = #---consumer key secret here--#


# Access Tokens
ACCESS_TOKEN = #---your access token here---# 
ACCESS_TOKEN_SECRET = #---your acess token secret here---# 

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

In [3]:
class TwitterData:
    def __init__(self, consumer_key, consumer_secret, ACCESS_TOKEN, ACCESS_TOKEN_SECRET):
        self.auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        self.auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        # initialzing the API
        self.api = tweepy.API(self.auth)
        
    def get_tweet_of(self, username):
        
        data = self.api.user_timeline(screen_name=username, count=200, tweet_mode='extended',)
        recent = min(data, key = lambda x:x.id).id
        tweets_ = data.copy()
        count = 0
        while len(tweets_)>0:
            try:
                tweets_ = api.user_timeline(screen_name=username, max_id=recent, count=200, tweet_mode='extended')
                more_recent = min(tweets_, key=lambda x: x.id).id
                if (recent==more_recent) or not tweets_:
                    break
                else:
                    recent = more_recent
                    data+=tweets_
                    
            except Exception as e:
                print(e)
                # In case of requsting more request than allowed in a window
                count += 1
                if count>2:
                    break
                time.sleep(120)
                continue
        # Dumping into JSON file
        with open('twitter_data.json', 'w+') as f:
            data_lst = []
            for tweet in data:
                data_lst.append(tweet._json)

            f.write(json.dumps(data_lst))
            
    # Helper functino for getting image count, returns None if not found.
    def _findImages(self, col):
        if pd.isnull(col):
            return
        if 'media' in col:
            media = col['media']
            imgcount = 0
            for i in media:
                # selecting only photo type, other available types are animated_vide/gif, video etc
                if i['type']=='photo':
                    imgcount+=1
            return imgcount
        else:
            return None

    # process tweets, as sated, returns data frame, save to csv as well
    def processTweets(self, save_csv=False):
        data = pd.read_json('twitter_data.json')
        columns = ['created_at', 'full_text', 'favorite_count', 'retweet_count']
        df = data[columns]
        df['ImageCount'] = data['extended_entities'].apply(self._findImages)
        # safety net
        df.drop_duplicates(inplace=True)
        if save_csv:
            df.to_csv('required_data.csv', index=False)
        return df

In [4]:
twd = TwitterData(consumer_key, consumer_secret, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
twd.get_tweet_of('midasIIITD')
data = twd.processTweets()

In [5]:
data.head()

,created_at,full_text,favorite_count,retweet_count,ImageCount
0,2019-04-09 16:45:07,RT @IIITDelhi: We are delighted to share that ...,0,9,NaN
1,2019-04-09 05:04:27,RT @Harvard: Professor Jelani Nelson founded A...,0,35,NaN
2,2019-04-09 05:04:11,RT @emnlp2019: For anyone interested in submit...,0,12,NaN
3,2019-04-08 19:38:09,RT @multimediaeval: Announcing the 2019 MediaE...,0,15,NaN
4,2019-04-08 07:08:12,"Many Congratulations to @midasIIITD student, S...",14,2,1.0


In [6]:
total_number_of_distinct_tweets = data.full_text.nunique()
print(total_number_of_distinct_tweets)

342


In [ ]:
dat